In [21]:
!pip install pyti

In [25]:
import time
import pytz
from datetime import datetime, timedelta
from dateutil import tz
from pandas_ta import rsi, ema, macd
from robin_stocks import robinhood as r
import yfinance as yf
import numpy as np
import pandas as pd
import configparser
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema
# from pyti.relative_strength_index import relative_strength_index as rsi
from ratelimit import limits, sleep_and_retry
# Set up logging
import logging
config = configparser.ConfigParser()
config.read('config/credentials.ini')
# Define variables from config
# Define variables from config
coins = [coin.strip() for coin in config['trading']['coins'].split(', ')]
stop_loss_percent = float(config['trading']['stop_loss_percent'])
percent_to_use = float(config['trading']['percent_to_use'])
verbose_mode = config['logging'].getboolean('verbose_mode')
debug_verbose = config['logging'].getboolean('debug_verbose')
reset_positions = config['logging'].getboolean('reset_positions')
minimum_usd_per_position = float(config['trading']['minimum_usd_per_position'])
pct_to_buy_with = float(config['trading']['percent_to_use'])
pct_to_buy_per_trade = float(config['trading']['percent_to_spend_per_trade'])
username = config['robinhood']['username']
password = config['robinhood']['password']
login = r.login(username, password)
print(f"Logged in as {username}")
# Function to get current price for a
logging.basicConfig(level=logging.INFO)





# Function to buy a coin
@sleep_and_retry
def buy_coin(coin_name, amount):
    try:
        current_price = get_current_price(coin_name)
        amount_to_buy = float(amount) / float(current_price)
        buying_usd = round(float(amount_to_buy), 2)
        buy_cost = round(float(buying_usd) * float(current_price), 2)
        result = r.orders.order_crypto(
            symbol=str(coin_name).upper(),
            amountIn='dollars',
            side='buy',
            quantityOrPrice=float(buy_cost),
            limitPrice=float(current_price),
            timeInForce='gtc',
            jsonify=True
        )
        return result
    except Exception as e:
        print(f"An error occurred when trying to buy {coin_name}: {e}")
        return None

# Function to calculate RSI
def calculate_rsi(data, period):
    # cast the numeric data in data to float types
    try:
        data = data.astype(float)
    except:
        print("Error converting data to float type")
        return None
    delta = data.diff()
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    average_gain = up.rolling(window=period).mean()
    average_loss = abs(down.rolling(window=period).mean())
    rs = average_gain / average_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Function to calculate EMA
def calculate_ema(data, window):
    return data.ewm(span=window, adjust=False).mean()

# Function to calculate MACD
def calculate_macd(data, short_window, long_window):
    short_ema = calculate_ema(data, short_window)
    long_ema = calculate_ema(data, long_window)
    macd_line = short_ema - long_ema
    signal_line = calculate_ema(macd_line, 9)
    return macd_line, signal_line

# Function to get local peaks
def get_local_peaks(data):
    peaks = argrelextrema(data, np.greater)
    return peaks

# Function to calculate percentage change
def calculate_percentage_change(old_price, new_price):
    return ((new_price - old_price) / old_price) * 100

# Function to get current price
def get_current_price(coin_name):
    return float(r.crypto.get_crypto_quote(coin_name)['mark_price'])

# Function to implement the stop loss strategy
@sleep_and_retry
def sell_coin(coin_name):
    current_price = float(get_current_price(coin_name))
    holdings = r.crypto.get_crypto_positions(info=None)
    coin_holdings = [coin for coin in holdings if coin['currency']['code'] == coin_name]
    amount_to_sell = float(coin_holdings[0]['quantity'])
    result = r.orders.order_crypto(
        symbol=coin_name,
        amountIn='quantity',
        side='sell',
        quantityOrPrice=float(amount_to_sell),
        limitPrice=float(current_price),
        timeInForce='gtc',
        jsonify=True
    )
    return result

# Function to implement the trailing stop loss strategy
def trailing_stop_loss(coin_name, percentage_drop):
    holdings = r.crypto.get_crypto_positions(info=None)
    coin_holdings = [coin for coin in holdings if coin['currency']['code'] == coin_name]
    if not coin_holdings:
        print(f"No holdings for {coin_name}")
        return
    initial_price = float(coin_holdings[0]['cost_bases'][0]['direct_cost_basis'])
    P_lowest = initial_price
    while True:
        current_price = get_current_price(coin_name)
        if current_price < P_lowest:
            P_lowest = current_price
        if current_price < P_lowest * (1 - percentage_drop/100):
            result = sell_coin(coin_name)
            print(f"Sold {coin_name} at {current_price} due to trailing stop loss.")
            return result
        time.sleep(300)  # sleep for 5 minutes

# Function to implement strategy 1
def strategy_1(coin):
    try:
        logging.info(f"Running strategy 1 for {coin}")

        # Get the RSI data
        price_data = r.crypto.get_crypto_historicals(coin, interval='hour', span='day')
        price_data = pd.DataFrame(price_data)
        price_data['close_price'] = price_data['close_price'].astype(float)
        rsi_data = calculate_rsi(price_data['close_price'], 14)

        # Check if RSI goes below 20
        if rsi_data[-1] < 20:
            logging.info(f"RSI for {coin} is below 20")

            # Watch for a RSI local peak that is above the lowest RSI
            rsi_local_high = get_local_peaks(rsi_data)

            # Monitor at five minute intervals until the RSI dips below the mean of the last five RSI values
            while True:
                rsi_data = calculate_rsi(price_data['close_price'], 14)
                if rsi_data[-1] < np.mean(rsi_data[-5:]):
                    logging.info(f"RSI for {coin} has dipped below the mean of the last five values")

                    # Trigger a market sell of the coin
                    sell_coin(coin)
                    break

            # If the difference between the first and last of the last three RSI measurements is greater than the standard distance between the last five
            if abs(rsi_data[-3] - rsi_data[-1]) > np.std(rsi_data[-5:]):
                logging.info(f"The difference between the first and last of the last three RSI measurements for {coin} is greater than the standard distance between the last five")

                # Trigger an immediate market sell of the coin
                sell_coin(coin)
    except Exception as e:
        logging.error(f"An error occurred when trying to run strategy 1 for {coin}: {e}")
# Function to implement strategy 2
def strategy_2(coin):
    try:
        # Get the RSI data
        rsi_data = calculate_rsi(coin, 14)

        # Check if RSI goes above 80
        if rsi_data[-1] > 80:
            # Trigger a limit sell of the coin
            sell_coin(coin)
    except Exception as e:
        logging.error(f"An error occurred when trying to run strategy 2 for {coin}: {e}")
# Function to calculate RSI Strength
@sleep_and_retry
def calculate_rsi_strength(coin_name):
    rsi_strength = 0.0
    rsi_prev = 0.0

    # Get the historical prices for the coin in the last 2 hours
    historical_prices = r.crypto.get_crypto_historicals(coin_name, interval='5minute', span='2hour')

    # Calculate the RSI for each price point
    close_prices = [float(data['close_price']) for data in historical_prices]
    rsi_values = RSI(close_prices)

    # Calculate the RSI strength
    for rsi in rsi_values:
        if rsi < rsi_low_limit and rsi_prev > rsi_high_limit:
            rsi_strength += 1.0
        rsi_prev = rsi

    return rsi_strength

# Function to implement the RSI strength strategy
@sleep_and_retry
def rsi_strength_strategy(coin_name):
    while True:
        rsi_strength = calculate_rsi_strength(coin_name)
        if rsi_strength > rsi_strength_threshold:
            sell_result = sell_coin(coin_name)
            print(f'Sold {coin_name} due to high RSI strength: {rsi_strength}')
            print(sell_result)
        time.sleep(300)  # Sleep for 5 minutes

def strategy3(coin):
    # # Strategy 3
    # 1. the rsi goes below 20
    # 2. monitor the macd at five minute intervals until
    # either of these scenarios could happen first:
    # scenario 1:
    # 1. The Macd (fast 12, slow 26, signal 9) crosses the signal line from below
    # 2. Trigger a market buy for the `buy_in` in USD of the coin at the current price.
    # scenario 2:
    # 1. rsi goes below 20 and the macd crosses the signal line from above
    # 2. trigger a market sell of the coin at the current price to sell all of the coin in holdings at the current price.
    try:
        # get the historical rsi data for the last 20 periods (a period is 5 minutes)
        rsi_data = calculate_rsi(coin, 14)
        # check if rsi goes below 20 during the last 20 periods
        # if any of the rsi values in the last 20 periods is below 20:
        if any(rsi < 20 for rsi in rsi_data):
            # then start monitoring the macd at five minute intervals
            while True:
                # get the current macd
                macd_line, signal_line = calculate_macd(coin, 12, 26)
                # get current rsi value
                rsi_data = calculate_rsi(coin, 14)
                # if the rsi goes above 70 then break out of the loop
                if rsi_data[-1] > 70:
                    break
                #^ Scenario 1
                # check if the macd line crosses the signal line from below 
                # (this is a bullish signal)
                if macd_line[-1] > signal_line[-1] and macd_line[-2] < signal_line[-2]:
                    # if it does, then trigger a market buy for the `buy_in` in USD of the coin at the current price.
                    buy_coin(coin)
                    break
                #^ Scenario 2
                # check if the macd line crosses the signal line from above
                # this is a bearish signal
                if macd_line[-1] < signal_line[-1] and macd_line[-2] > signal_line[-2]:
                    # if it does, then trigger a market sell of the coin at the current price to sell all of the coin in holdings at the current price.
                    sell_coin(coin)
                    break
        elif rsi_data[-1] > 80:
            sell_coin(coin)
            time.sleep(300)  # Sleep for 5 minutes
    except Exception as e:
        logging.error(f"An error occurred when trying to run strategy 3 for {coin}: {e}")
def trailing_stop_loss(coin, percent):
    # # Strategy 5 - Trailing Stop Loss
    # if a coin's price is down by more than 5% in the last 24 hours at any point then begin a five-minute tracking cycle of the coin's price.
    # - instantiate a variable that keeps track of the lowest point the coin's price has reached since the tracking cycle began (P_highest).
    # - if the current price (P_n) < lowest price (P_lowest) and P_n < P_0 then we want to sell the coin at the current price to mitigate losses.

    # get the historical prices for the coin in the last 24 hours
    historical_prices = r.crypto.get_crypto_historicals(coin, interval='5minute', span='day')
    #todo cast all the close prices to floats
    # calculate the percent change in price from the first price to the last price
    percent_change = (float(historical_prices[-1]['close_price']) - float(historical_prices[0]['close_price'])) / float(historical_prices[0]['close_price'])
    # if the percent change is less than -5% then start tracking the coin's price
    if percent_change < -0.05:
        # start a while loop to track the price of the coin every five minutes
        while True:
            # get the current price of the coin
            current_price = float(r.crypto.get_crypto_quote(coin)['mark_price'])
            # if the current price is greater than the highest price then update the highest price
            if current_price > highest_price:
                highest_price = current_price
            # if the current price is less than the highest price and the current price is less than the original price then sell the coin
            if current_price < highest_price and current_price < float(historical_prices[0]['close_price']):
                sell_coin(coin)
                break
            time.sleep(300)
# start a while loop to continuously check the price of the coin, and implement all the strategies
import schedule
import time

def job():
    strategy_1('BTC')
    print('Strategy 1 complete')
    strategy_2('BTC')
    print('Strategy 2 complete')
    trailing_stop_loss('BTC', 5)
    print('Strategy 5 complete')
count = 0
while True:
    count += 1
    job()
    schedule.every(5).minutes.do(job)
    print(f'Iteration {count}')
    schedule.run_pending()
    time.sleep(1)
    # run job() function every 5 minutes




INFO:root:Running strategy 1 for BTC


Logged in as "graham.waters37@gmail.com"


ERROR:root:An error occurred when trying to run strategy 1 for BTC: -1
ERROR:root:An error occurred when trying to run strategy 2 for BTC: 'NoneType' object is not subscriptable


Strategy 1 complete
Error converting data to float type
Strategy 2 complete
Strategy 5 complete
Iteration 1


INFO:root:Running strategy 1 for BTC
ERROR:root:An error occurred when trying to run strategy 1 for BTC: -1
ERROR:root:An error occurred when trying to run strategy 2 for BTC: 'NoneType' object is not subscriptable


...

INFO:root:Running strategy 1 for BTC
ERROR:root:An error occurred when trying to run strategy 1 for BTC: -1
ERROR:root:An error occurred when trying to run strategy 2 for BTC: 'NoneType' object is not subscriptable


Error converting data to float type


INFO:root:Running strategy 1 for BTC
ERROR:root:An error occurred when trying to run strategy 1 for BTC: -1
ERROR:root:An error occurred when trying to run strategy 2 for BTC: 'NoneType' object is not subscriptable


Error converting data to float type


KeyboardInterrupt: 

In [28]:
import time
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.signal import argrelextrema
from pandas_ta import rsi, macd
from robin_stocks import robinhood as r
from ratelimit import limits, sleep_and_retry
import configparser

# Set up logging
import logging
logging.basicConfig(level=logging.INFO)

# Read variables from config
config = configparser.ConfigParser()
config.read('config/credentials.ini')

coins = [coin.strip() for coin in config['trading']['coins'].split(', ')]
buy_in_usd = float(config['trading']['minimum_usd_per_trade'])

# Function to buy a coin
@sleep_and_retry
def buy_coin(coin_name, amount):
    try:
        current_price = get_current_price(coin_name)
        amount_to_buy = float(amount) / float(current_price)
        buying_usd = round(float(amount_to_buy), 2)
        buy_cost = round(float(buying_usd) * float(current_price), 2)
        result = r.orders.order_crypto(
            symbol=str(coin_name).upper(),
            amountIn='dollars',
            side='buy',
            quantityOrPrice=float(buy_cost),
            limitPrice=float(current_price),
            timeInForce='gtc',
            jsonify=True
        )
        return result
    except Exception as e:
        print(f"An error occurred when trying to buy {coin_name}: {e}")
        return None

# Function to sell a coin with limit sell
@sleep_and_retry
def sell_coin(coin_name, amount):
    try:
        current_price = get_current_price(coin_name)
        amount_to_sell = float(amount)
        sell_value = round(float(amount_to_sell) * float(current_price), 2)
        result = r.orders.order_crypto(
            symbol=str(coin_name).upper(),
            amountIn='quantity',
            side='sell',
            quantityOrPrice=float(amount_to_sell),
            limitPrice=float(current_price),
            timeInForce='gtc',
            jsonify=True
        )
        return result
    except Exception as e:
        print(f"An error occurred when trying to sell {coin_name}: {e}")
        return None
# Function to calculate RSI
def calculate_rsi(data, period):
    # cast the numeric data in data to float types
    try:
        data = data.astype(float)
    except:
        print("Error converting data to float type")
        return None
    delta = data.diff()
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    average_gain = up.rolling(window=period).mean()
    average_loss = abs(down.rolling(window=period).mean())
    rs = average_gain / average_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Function to calculate MACD
def calculate_macd(data, short_window, long_window):
    short_ema = data.ewm(span=short_window, adjust=False).mean()
    long_ema = data.ewm(span=long_window, adjust=False).mean()
    macd_line = short_ema - long_ema
    signal_line = macd_line.ewm(span=9, adjust=False).mean()
    return macd_line, signal_line

# Function to get local peaks
def get_local_peaks(data):
    peaks = argrelextrema(data, np.greater)
    return peaks

# Function to get current price
def get_current_price(coin_name):
    return float(r.crypto.get_crypto_quote(coin_name)['mark_price'])

# Function to implement strategy
def strategy(coin):
    try:
        # Get the historical data
        data = yf.download(coin, start='2020-01-01', end='2022-12-31')

        # Calculate MACD
        macd_line, signal_line = calculate_macd(data['Close'], 12, 26)

        # Calculate RSI
        data['RSI'] = rsi(data['Close'], 14)

        # Initialize holding status
        holding = False

        # Iterate over data
        for i in range(1, len(data)):
            # Check if RSI is below 20
            if data['RSI'].iloc[i] < 20:
                # Check if MACD line crosses signal line from below
                if macd_line.iloc[i] > signal_line.iloc[i] and macd_line.iloc[i-1] < signal_line.iloc[i-1]:
                    if holding:
                        print(f"Sell {coin} at {data['Close'].iloc[i]}")
                        holding = False
                # Check if MACD line crosses signal line from above
                elif macd_line.iloc[i] < signal_line.iloc[i] and macd_line.iloc[i-1] > signal_line.iloc[i-1]:
                    if holding:
                        print(f"Sell {coin} at {data['Close'].iloc[i]}")
                        holding = False
                else:
                    if not holding:
                        print(f"Buy {coin} at {data['Close'].iloc[i]}")
                        holding = True
    except Exception as e:
        logging.error(f"An error occurred when trying to run strategy for {coin}: {e}")



In [34]:
# List of coins to trade
from tqdm import tqdm
coins = config['trading']['coins'].split(', ')



In [ ]:
def strategy():
    # if the macd line crosses above the signal line, and rsi has recently been below 30, buy
    if macd > signal and rsi < 30:
        order_target_percent(security, 1)
    # if the macd line crosses below the signal line, and rsi has recently been above 70, sell
    elif macd < signal and rsi > 70:
        order_target_percent(security, 0)
    # otherwise, do nothing
    else:
    

In [35]:

while True:
    for coin in tqdm(coins):
        # Buy-in USD amount
        buy_coin(coin, buy_in_usd)
        time.sleep(5)  # Add a small delay to avoid Robinhood rate limits
        # Sell at 100% of holdings with limit sell
        holdings = r.crypto.get_crypto_positions(info=None)
        coin_holdings = [coin for coin in holdings if coin['currency']['code'] == coin]
        if coin_holdings:
            amount_to_sell = float(coin_holdings[0]['quantity'])
            sell_coin(coin, amount_to_sell)
    time.sleep(300)  # Sleep for 5 minutes


 36%|███▌      | 5/14 [00:29<00:53,  5.97s/it]

KeyboardInterrupt: 